In [4]:
## Importing Libraries
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle

import joblib
import pickle
from sklearn.utils import shuffle

# from xgboost import XGBClassifier
from tensorflow.keras import Sequential,layers, losses, optimizers
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.models import load_model
import time

In [5]:
## Loading the CICIDS2017 training dataset
df_train = pd.read_csv("all_group_train_normalized.csv")

In [6]:
df_train['Label'].value_counts()

Benign          53432
DoS              8906
Web Attack       8906
Infiltration     8905
Port Scan        8905
DDoS             8905
Brute Force      8905
Name: Label, dtype: int64

In [7]:
## Loading the CICIDS2017 testing dataset
df_test = pd.read_csv("all_group_test_normalized.csv")

In [8]:
## Extracting attacks from the testing data
infiltration_attack = df_test[df_test['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(infiltration_attack.shape)
portscan_attack = df_test[df_test['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(portscan_attack.shape)
ddos_attack = df_test[df_test['Label'] == 'DDoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(ddos_attack.shape)
bruteforce_attack = df_test[df_test['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(bruteforce_attack.shape)
dos_attack = df_test[df_test['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(dos_attack.shape)
web_attack = df_test[df_test['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(web_attack.shape)
benign_data = df_test[df_test['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(benign_data.shape)

(3817, 1525)
(3817, 1525)
(3817, 1525)
(3817, 1525)
(3816, 1525)
(3816, 1525)
(22900, 1525)


In [10]:
## Loading the CICIDS 2018 training data
df_test_2018 = pd.read_csv("all_group_train_normalized_2018.csv")

In [11]:
df_test_2018 = df_test_2018.sample(frac = 1).reset_index(drop = True)

In [12]:
## Loading the botnet data from CICIDS2017 emulating Zero day attack
bot = pd.read_csv("botnet_train_normalized.csv")
bot_test = pd.read_csv("botnet_test_normalized.csv")

In [13]:
botnet = bot[bot['Label'] == 'Botnet'].drop(['Label'], axis=1).iloc[:,0:1525]
botnet_test = bot_test[bot_test['Label'] == 'Botnet'].drop(['Label'], axis=1).iloc[:,0:1525]

In [14]:
X_train_bot = bot.iloc[:,0:1525]
Y_train_bot = bot.iloc[:,1525:1526]

In [15]:
X_train_2018 = df_test_2018.iloc[:,0:1525]
Y_train_2018 = df_test_2018.iloc[:,1526:1527]

In [16]:
X_train_2017 = df_test.iloc[:,0:1525]
Y_train_2017 = df_test.iloc[:,1526:1527]

In [19]:
Y_train_bot.columns = ['Label_binary']

In [20]:
Y_train_bot.loc[Y_train_bot['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_bot.loc[Y_train_bot['Label_binary'] == 'Botnet', 'Label_binary'] = 1

<ipython-input-20-7e9813a6ce09>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train_bot.loc[Y_train_bot['Label_binary'] == 'Benign', 'Label_binary'] = 0
<ipython-input-20-7e9813a6ce09>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train_bot.loc[Y_train_bot['Label_binary'] == 'Botnet', 'Label_binary'] = 1


In [21]:
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [22]:
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [23]:
X_train_total = pd.concat([X_train_2017,X_train_2018,X_train_bot,X_train_bot])
Y_train_total = pd.concat([Y_train_2017,Y_train_2018, Y_train_bot,Y_train_bot])

In [24]:
X_train_ = X_train_total.values.astype('float32')
Y_train_ = Y_train_total.values.astype('float32')

In [25]:
## Extracting attacks from the CICIDS2018 dataset
infiltration_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(infiltration_attack_2018.shape)
portscan_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(portscan_attack_2018.shape)
ddos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'ddos'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(ddos_attack_2018.shape)
bruteforce_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(bruteforce_attack_2018.shape)
dos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(dos_attack_2018.shape)
web_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(web_attack_2018.shape)
benign_data_2018 = df_test_2018[df_test_2018['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:1525]
print(benign_data_2018.shape)

(29116, 1525)
(0, 1525)
(29115, 1525)
(29116, 1525)
(29116, 1525)
(29116, 1525)
(145579, 1525)


In [26]:
## Prediction Function
def predictions(y_test):
    pred_list = []
    for i in y_test:
        if i > 0.5:
            pred = [1]
        else:
            pred = [0]

        pred_list.append(pred)
    return pred_list

In [27]:
## Calculates detection accuracy
def accuracy_calc(inf):
    inf_ = predictions(inf)
    cnt = 0
    for i in inf_:
        if i[0] != 1:
            cnt += 1
    acc = 1 - (cnt / len(inf))
    return cnt, acc

In [29]:
## Loading the Stage 1 baseline DNN
stage1_DNN = tf.keras.models.load_model('Stage1_DNN.h5', compile = False)

In [ ]:
## Testing the Baseline DNN against unseen CICIDS 2017 test data

In [ ]:
inf = stage1_DNN.predict(infiltration_attack.values.astype('float32'))

In [ ]:
cnt_inf, acc_inf = accuracy_calc(inf)
print(cnt_inf)
print(acc_inf)

In [ ]:
port = stage1_DNN.predict(portscan_attack.values.astype('float32'))

In [ ]:
cnt_port, acc_port = accuracy_calc(port)
print(cnt_port)
print(acc_port)

In [ ]:
dos = stage1_DNN.predict(dos_attack.values.astype('float32'))

In [ ]:
cnt_dos, acc_dos = accuracy_calc(dos)
print(cnt_dos)
print(acc_dos)

In [ ]:
ddos = stage1_DNN.predict(ddos_attack.values.astype('float32')) 

In [ ]:
cnt_ddos, acc_ddos = accuracy_calc(ddos)
print(cnt_ddos)
print(acc_ddos)

In [ ]:
web = stage1_DNN.predict(web_attack.values.astype('float32'))

In [ ]:
cnt_web, acc_web = accuracy_calc(web)
print(cnt_web)
print(acc_web)

In [ ]:
brute = stage1_DNN.predict(bruteforce_attack.values.astype('float32'))

In [ ]:
cnt_brute, acc_brute = accuracy_calc(brute)
print(cnt_brute)
print(acc_brute)

In [ ]:
ben = stage1_DNN.predict(benign_data.values.astype('float32'))

In [ ]:
cnt_benign, acc_benign = accuracy_calc(ben)
print(cnt_benign)
print(1-acc_benign)

In [ ]:
### Testing the Baseline DNN against CICIDS 2018 data

In [ ]:
inf_2018 = stage1_DNN.predict(infiltration_attack_2018)
dos_2018 = stage1_DNN.predict(dos_attack_2018)
ddos_2018 = stage1_DNN.predict(ddos_attack_2018)
web_2018 = stage1_DNN.predict(web_attack_2018)
brute_2018 = stage1_DNN.predict(bruteforce_attack_2018)
ben_2018 = stage1_DNN.predict(benign_data_2018)

In [ ]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf_2018)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos_2018)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos_2018)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web_2018)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute_2018)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben_2018)
print(cnt_ben)
print(1-acc_ben)

## Retraining With new data (CICIDS2018)

In [30]:
## Layers that we selected can be modified from the base DNN
mod_layers = [2,3]

In [31]:
## Neurons that are not trainable in the modifiable layers
layer2_ind = [h for h in range(0,64,3)]
layer3_ind = [h for h in range(0,32,3)]

In [32]:
## Initializing the new model
retrained_stage1 = tf.keras.Sequential()

In [33]:
## Setting up the architecture of the new retrained model
for i, layer in enumerate(stage1_DNN.layers):
    if i not in mod_layers:
        layer.trainable = False
        retrained_stage1.add(layer)
    else:
        retrained_stage1.add(layer)
        if i == 2:
            for ind in range(0,64):
                if ind in layer2_ind:
                    retrained_stage1.layers[-1].kernel[ind]._trainable = False
        if i == 3:
            for ind in range(0,32):
                if ind in layer3_ind:
                    retrained_stage1.layers[-1].kernel[ind]._trainable = False
            retrained_stage1.add(Dense(32, activation='relu'))
        


In [34]:
retrained_stage1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1525)              2327150   
_________________________________________________________________
dense_8 (Dense)              (None, 128)               195328    
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 2,533,903
Trainable params: 11,392
Non-trainable params: 2,522,511
_________________________________________

In [35]:
## Compilation and training of the new retrained model
start_time = time.time()
retrained_stage1.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
retrained_stage1.fit(X_train_ , Y_train_, batch_size=256, epochs=50, shuffle=True)
end_time = time.time()

Epoch 1/50
1369/1369 [==============================] - 8s 6ms/step - loss: 0.2929 - accuracy: 0.9004
Epoch 2/50
1369/1369 [==============================] - 7s 5ms/step - loss: 0.1766 - accuracy: 0.9319
Epoch 3/50
1369/1369 [==============================] - 7s 5ms/step - loss: 0.1566 - accuracy: 0.9402
Epoch 4/50
1369/1369 [==============================] - 8s 6ms/step - loss: 0.1462 - accuracy: 0.9439
Epoch 5/50
1369/1369 [==============================] - 7s 5ms/step - loss: 0.1383 - accuracy: 0.9469
Epoch 6/50
1369/1369 [==============================] - 8s 6ms/step - loss: 0.1324 - accuracy: 0.9490
Epoch 7/50
1369/1369 [==============================] - 8s 5ms/step - loss: 0.1275 - accuracy: 0.9508
Epoch 8/50
1369/1369 [==============================] - 8s 6ms/step - loss: 0.1231 - accuracy: 0.9525
Epoch 9/50
1369/1369 [==============================] - 8s 6ms/step - loss: 0.1197 - accuracy: 0.9538
Epoch 10/50
1369/1369 [==============================] - 8s 6ms/step - loss: 0.115

In [46]:
## Saving the new model
retrained_stage1.save('retrained_stage1_Hybrid.h5')

## Testing the retrained model against zero-day and CICIDS 2018 data

In [ ]:
time_taken = end_time - start_time
print(time_taken)

In [42]:
bot_ = retrained_stage1.predict(botnet_test.values.astype('float32'))

In [43]:
cnt_bot, acc_bot = accuracy_calc(bot_)
print(cnt_bot)
print(acc_bot)

14
0.9902506963788301


In [ ]:
### CICIDS2018 Testing with retrained model

In [36]:
inf_2018 = retrained_stage1.predict(infiltration_attack_2018)
dos_2018 = retrained_stage1.predict(dos_attack_2018)
ddos_2018 = retrained_stage1.predict(ddos_attack_2018)
web_2018 = retrained_stage1.predict(web_attack_2018)
brute_2018 = retrained_stage1.predict(bruteforce_attack_2018)
ben_2018 = retrained_stage1.predict(benign_data_2018)

In [37]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf_2018)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos_2018)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos_2018)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web_2018)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute_2018)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben_2018)
print(cnt_ben)
print(1-acc_ben)

Infiltration
2753
0.9054471768100014
DoS
387
0.9867083390575628
DDoS
24
0.9991756826378155
Web
447
0.9846476164308284
Brute
285
0.9902115675230114
Benign
140986
0.9684501198661895


In [ ]:
## CICIDS2017 Testing

In [44]:
port = retrained_stage1.predict(portscan_attack)
inf = retrained_stage1.predict(infiltration_attack)
dos = retrained_stage1.predict(dos_attack)
ddos = retrained_stage1.predict(ddos_attack)
web = retrained_stage1.predict(web_attack)
brute = retrained_stage1.predict(bruteforce_attack)
ben = retrained_stage1.predict(benign_data)

In [45]:
print('Portscan')
cnt_port, acc_port = accuracy_calc(port)
print(cnt_port)
print(acc_port)
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben)
print(cnt_ben)
print(1-acc_ben)

Portscan
4
0.9989520565889441
Infiltration
41
0.989258580036678
DoS
221
0.9420859538784067
DDoS
71
0.9813990044537595
Web
126
0.9669811320754716
Brute
145
0.9620120513492272
Benign
22769
0.9942794759825327
